In [1]:
import os 
import re
import json 
import pandas as pd
import traceback

In [2]:

import langchain
from langchain.chat_models import ChatOpenAI
from langchain_community.llms import HuggingFaceEndpoint
from langchain.chains import LLMChain, SequentialChain, ConversationChain
from langchain_community.callbacks import get_openai_callback
# from langchain_openai import OpenAI
from langchain_core.prompts import PromptTemplate
from langchain.agents import AgentType
from langchain.agents import load_tools 
from langchain.agents import initialize_agent
import PyPDF2

In [3]:
from dotenv import load_dotenv
load_dotenv()
openai_key = os.getenv("OPENAI_API_KEY")
HUGGINGFACEHUB_API_TOKEN = os.getenv("HUGGINGFACEHUB_API_TOKEN")
serpapi_key = os.getenv("serpapi_key")
os.environ['HUGGINGFACEHUB_API_TOKEN']=HUGGINGFACEHUB_API_TOKEN 

## Test HUGGINGFACE with google-search-results

In [4]:
question = "Who won the FIFA World Cup in the year 2022? "

template = """Question: {question}

Answer: Let's think step by step."""

prompt = PromptTemplate.from_template(template)

repo_id =  "mistralai/Mistral-7B-Instruct-v0.2"

llm = HuggingFaceEndpoint(
    repo_id=repo_id, max_length=128, temperature=0.5, token=HUGGINGFACEHUB_API_TOKEN
)
llm_chain = LLMChain(prompt=prompt, llm=llm)
print(llm_chain.run(question))
tool = load_tools(["serpapi"],serpapi_api_key=serpapi_key,llm=llm)
agent = initialize_agent(tool, llm,agent = AgentType.ZERO_SHOT_REACT_DESCRIPTION,verbose=True)
agent.run("world cup 2022 winner")

WARNING! max_length is not default parameter.
                    max_length was transferred to model_kwargs.
                    Please make sure that max_length is what you intended.
WARNING! token is not default parameter.
                    token was transferred to model_kwargs.
                    Please make sure that token is what you intended.
d:\GenAI\MCQgen\env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to C:\Users\Abdel\.cache\huggingface\token
Login successful


d:\GenAI\MCQgen\env\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(
d:\GenAI\MCQgen\env\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


 The FIFA World Cup is a soccer (or football, depending on where you're from) tournament that takes place every four years. It's the most prestigious competition in the sport. However, the last World Cup was held in Qatar in 2022, and the next one is scheduled for 2026. So, as of now, no team has won the FIFA World Cup in the year 2022. We'll have to wait until the tournament actually takes place to find out who the champions will be.


d:\GenAI\MCQgen\env\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
 I don't know the answer to this question, I should search for it.
Action: Search
Action Input: world cup 2022 winner
Observation: Argentina national football team
Thought: I now know the final answer
Final Answer: Argentina national football team

> Finished chain.


'Argentina national football team'

In [5]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}


In [6]:
TEMPLATE = """
Text:{text}
You are an expert MCQ Marker. Given the above text, it is your job to create a Quiz of {number} multiple choice questions
for {subject} students in {tone} tone.
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like RESPONSE_JSON below and use it as a guide.
Ensure to make {number} MCQs.
### RESPONSE_JSON 
{response_json}

"""

quiz_gen_prompt = PromptTemplate(
    input_variables = ["text", "number", "subject","tone","response_json"],
    template = TEMPLATE
)

In [7]:
quiz_chain = LLMChain(prompt=quiz_gen_prompt, llm=llm,output_key='quiz',verbose=True)


In [8]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [9]:
quiz_eval_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE)

review_chain=LLMChain(llm=llm, prompt=quiz_eval_prompt, output_key="review", verbose=True)


In [10]:
gen_eval_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)

In [11]:
file_path = r"D:\GenAI\MCQgen\data.txt"

In [12]:
with open(file_path, 'r') as file:
    TEXT = file.read()

In [13]:
print(TEXT)

Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]

Biologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each defined by the nature of their research questions and the tools that they use.[7][8][9] Like other scientists, bio

In [14]:
# Serialize the Python dictionary into a JSON-formatted string
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [15]:
NUMBER=5
SUBJECT="biology"
TONE="simple"
#https://python.langchain.com/docs/modules/model_io/llms/token_usage_tracking

#How to setup Token Usage Tracking in LangChain
with get_openai_callback() as cb:
    response=gen_eval_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )

d:\GenAI\MCQgen\env\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]

Biologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each define

In [16]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:0
Prompt Tokens:0
Completion Tokens:0
Total Cost:0.0


In [17]:
response

{'text': 'Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]\n\nBiologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each defined by the nature of their research questions and the tools that they use.[7][8][9] Like other sci

In [18]:
quiz=response.get("quiz")
quiz

'### RESPONSE_JSON\n{\n"1": {\n"mcq": "What is the scientific study of?",\n"options": {\n"a": "The study of the physical world",\n"b": "The study of the origins of the universe",\n"c": "The scientific study of life",\n"d": "The study of the properties of matter"\n},\n"correct": "c"\n},\n"2": {\n"mcq": "Which of the following is a major theme in the study of biology?",\n"options": {\n"a": "The study of the properties of matter",\n"b": "The study of the physical world",\n"c": "The scientific study of life, with a focus on the unity and diversity of life",\n"d": "The study of the origins of the universe"\n},\n"correct": "c"\n},\n"3": {\n"mcq": "Which of the following is true about organisms?",\n"options": {\n"a": "Organisms do not have cells",\n"b": "Organisms do not have hereditary information",\n"c": "All organisms are able to regulate their own internal environments",\n"d": "Organisms do not process energy"\n},\n"correct": "a"\n},\n"4": {\n"mcq": "Which of the following is a major subd

In [20]:
quiz=json.loads(quiz[quiz.index("{"):len(quiz)-quiz[::-1].index("}")] )
# print(quiz[quiz.index("{"):len(quiz)-quiz[::-1].index("}")] + "}" * (quiz.count('{')-quiz.count('}')))
# print(quiz)

In [21]:
quiz

{'1': {'mcq': 'What is the scientific study of?',
  'options': {'a': 'The study of the physical world',
   'b': 'The study of the origins of the universe',
   'c': 'The scientific study of life',
   'd': 'The study of the properties of matter'},
  'correct': 'c'},
 '2': {'mcq': 'Which of the following is a major theme in the study of biology?',
  'options': {'a': 'The study of the properties of matter',
   'b': 'The study of the physical world',
   'c': 'The scientific study of life, with a focus on the unity and diversity of life',
   'd': 'The study of the origins of the universe'},
  'correct': 'c'},
 '3': {'mcq': 'Which of the following is true about organisms?',
  'options': {'a': 'Organisms do not have cells',
   'b': 'Organisms do not have hereditary information',
   'c': 'All organisms are able to regulate their own internal environments',
   'd': 'Organisms do not process energy'},
  'correct': 'a'},
 '4': {'mcq': 'Which of the following is a major subdiscipline of biology?',


In [22]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [23]:
quiz_table_data

[{'MCQ': 'What is the scientific study of?',
  'Choices': 'a: The study of the physical world | b: The study of the origins of the universe | c: The scientific study of life | d: The study of the properties of matter',
  'Correct': 'c'},
 {'MCQ': 'Which of the following is a major theme in the study of biology?',
  'Choices': 'a: The study of the properties of matter | b: The study of the physical world | c: The scientific study of life, with a focus on the unity and diversity of life | d: The study of the origins of the universe',
  'Correct': 'c'},
 {'MCQ': 'Which of the following is true about organisms?',
  'Choices': 'a: Organisms do not have cells | b: Organisms do not have hereditary information | c: All organisms are able to regulate their own internal environments | d: Organisms do not process energy',
  'Correct': 'a'},
 {'MCQ': 'Which of the following is a major subdiscipline of biology?',
  'Choices': 'a: Physics | b: Chemistry | c: Molecular biology | d: Anatomy and physio

In [24]:
quiz=pd.DataFrame(quiz_table_data)

In [25]:
quiz.to_csv("mcq_ml.csv",index=False)

In [26]:
from datetime import datetime
datetime.now().strftime('%m_%d_%Y_%H_%M_%S')

'05_05_2024_00_46_24'